# Random Search e Nested Cross-Validation per ottimizzare gli iperparametri del classificatore LightGBM

# Import

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import optuna


# Caricamento dati di training 

In [ ]:
BASE = Path(__file__).parent
X = pd.read_csv(BASE / 'train_values.csv', index_col=0)
y_df = pd.read_csv(BASE / 'train_labels.csv', index_col=0)
y = y_df['label'] if 'label' in y_df.columns else y_df.iloc[:, 0]


# Parte di preprocessing

In [ ]:
cat_cols = X.select_dtypes(include='object').columns.tolist()
for c in cat_cols:
    X[c] = X[c].astype('category')

num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
i = SimpleImputer(strategy='median')
X[num_cols] = i.fit_transform(X[num_cols])

# Clipping dei valori anomali ai percentili 1° e 99°
# Serve a limitare l'influenza degli outlier estremi nelle variabili numeriche
for col in num_cols:
    lo, hi = X[col].quantile(0.01), X[col].quantile(0.99)
    X[col] = X[col].clip(lo, hi)

# Aggiunge una nuova feature che indica il numero di valori mancanti per ogni riga
# Utile per informare il modello su possibili anomalie nei dati
X['missing_count'] = X.isnull().sum(axis=1)

# Imposta il trasformatore di colonne per il preprocessing delle variabili numeriche e categoriche
dct = ColumnTransformer([
    ('num', SimpleImputer(strategy='median'), num_cols),     
    ('scale', StandardScaler(), num_cols),                  
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols) 
])

# Applica il preprocessing ai dati di training
X_proc = dct.fit_transform(X)

 # Nested-Cross-validation per la ricerca degli iperparametri migliori 

In [ ]:
OUTER_K, INNER_K, N_TRIALS = 5, 3, 40
outer_cv = StratifiedKFold(n_splits=OUTER_K, shuffle=True, random_state=42)
inner_cv = StratifiedKFold(n_splits=INNER_K, shuffle=True, random_state=2025)

# Definisce la funzione obiettivo per Optuna per l'ottimizzazione degli iperparametri di LightGBM
def objective(trial, X_tr, y_tr):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 700, 1300),           # Numero di alberi nella foresta
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),       # Tasso di apprendimento
        'num_leaves': trial.suggest_int('num_leaves', 120, 200),                # Numero massimo di foglie in un albero
        'min_child_samples': trial.suggest_int('min_child_samples', 30, 100),   # Minimo numero di dati in una foglia
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),                # Percentuale di campioni usati per ciascun albero
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),  # Percentuale di colonne usate per ciascun albero
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 0.9),  # Percentuale di feature da considerare in ogni iterazione
        'reg_alpha': trial.suggest_float('reg_alpha', 0.2, 0.8),                # Regolarizzazione L1
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 0.8),             # Regolarizzazione L2
        'random_state': 42,                                                     # Semina per la riproducibilità
        'n_jobs': -1,                                                           # Usa tutti i core disponibili
        'verbose': -1                                                           # Disattiva output verboso
    }
    scores = []
    # Cross-validation interna per valutare le performance dei parametri selezionati
    for tr_idx, va_idx in inner_cv.split(X_tr, y_tr):
        X_t, X_v = X_tr[tr_idx], X_tr[va_idx]
        y_t, y_v = y_tr[tr_idx], y_tr[va_idx]
        model = LGBMClassifier(**params)
        model.fit(X_t, y_t)
        preds = model.predict(X_v)
        scores.append(f1_score(y_v, preds, average='micro'))
    return np.mean(scores)

# Raccolta metriche
metrics_per_fold = {
    'accuracy': [],
    'precision_macro': [],
    'recall_macro': [],
    'f1_micro': [],
    'f1_macro': [],
}

# Nested CV
for fold, (tr_idx, va_idx) in enumerate(outer_cv.split(X_proc, y), 1):
    print(f"\nFold {fold}/{OUTER_K}")
    X_tr, X_va = X_proc[tr_idx], X_proc[va_idx]
    y_tr, y_va = y[tr_idx], y[va_idx]

    study = optuna.create_study(direction='maximize')
    study.optimize(lambda t: objective(t, X_tr, y_tr), n_trials=N_TRIALS)

    best = study.best_params
    model = LGBMClassifier(**best)
    model.fit(X_tr, y_tr)
    y_pred = model.predict(X_va)

    acc = accuracy_score(y_va, y_pred)
    prec = precision_score(y_va, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_va, y_pred, average='macro', zero_division=0)
    f1_mi = f1_score(y_va, y_pred, average='micro')
    f1_ma = f1_score(y_va, y_pred, average='macro')

    metrics_per_fold['accuracy'].append(acc)
    metrics_per_fold['precision_macro'].append(prec)
    metrics_per_fold['recall_macro'].append(rec)
    metrics_per_fold['f1_micro'].append(f1_mi)
    metrics_per_fold['f1_macro'].append(f1_ma)

    print(f"Acc: {acc:.4f} | Prec_macro: {prec:.4f} | Recall_macro: {rec:.4f} | F1_micro: {f1_mi:.4f} | F1_macro: {f1_ma:.4f}")

# Report finale
print("\n=== Risultati medi sui fold esterni ===")
for metric, values in metrics_per_fold.items():
    print(f"{metric}: {np.mean(values):.4f} ± {np.std(values):.4f}")